<a href="https://colab.research.google.com/github/stevedepp/TransferLearningCatsDogs2ASL/blob/main/1_keras_custom_classifier_with_transfer_learning_100_epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1_keras_custom_classifier_with_transfer_learning_100_epochs.ipynb**

This version runs the model 100 epochs on the hands data which is sourced from my google drive.

<table class="tfo-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/practicaldl/Practical-Deep-Learning-Book/blob/master/code/chapter-3/1-keras-custom-classifier-with-transfer-learning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/practicaldl/Practical-Deep-Learning-Book/blob/master/code/chapter-3/1-keras-custom-classifier-with-transfer-learning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This code is part of [Chapter 3: Cats versus Dogs: Transfer Learning in 30 Lines with Keras](https://learning.oreilly.com/library/view/practical-deep-learning/9781492034858/ch03.html).

Note: In order to run this notebook on Google Colab you need to [follow these instructions](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb#scrollTo=WzIRIt9d2huC) so that the local data such as the images are available in your Google Drive.

# Building a Custom Classifier in Keras with Transfer Learning

As promised, it’s time to build our state of the art classifier in 30 lines or fewer! At a high level, we will follow the steps shown below:

- **Organize the data**: Download labeled images of cats and dogs from Kaggle. Then divide the images into training and validation folders.
- **Set up the configuration**: Define a pipeline for reading data, including preprocessing the images (e.g. resizing) and batching multiple images together.
- **Load and augment the data**: In the absence of a ton of training images, make small changes (augmentation) like rotation, zooming, etc to increase variation in training data.
- **Define the model**: Take a pre-trained model, remove the last few layers, and append a new classifier layer. Freeze the weights of original layers (i.e. make them unmodifiable). Select an optimizer algorithm and a metric to track (like accuracy).
- **Train and test**: Start training for a few iterations. Save the model to eventually load inside any application for predictions.

## Organize the data

Before training, we need to store our [downloaded dataset](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/train.zip) in the right folder structure. Remember to make the `data` directory where we will be performing the refactoring. We’ll divide the images into two sets – training and validation. Our directory structure will look something like this: 

```
data
 |__train
 |    |__cat
 |    |__dog
 |__val
      |__cat
      |__dog
```

In Linux/Mac, the following lines of command can help achieve this directory structure:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
path_data = '/content/gdrive/My Drive/data/dog_cat/'

In [ ]:
!ls -l /content/gdrive/My\ Drive/data/dog_cat

total 8
drwx------ 2 root root 4096 Jun  3 23:32 train
drwx------ 2 root root 4096 Jun  3 23:32 val


In [ ]:
#!unzip train.zip
#%mv train data
#%cd data
#%mkdir train val
#%mkdir train/cat train/dog
#%mkdir val/cat val/dog

The 25,000 files inside the data folder are prefixed with ‘cat’ and ‘dog’. Now, move the files into their respective directories. To keep our initial experiment short, we’ll pick 250 random files per class and place them in training and validation folders. You can increase/decrease this number anytime, to experiment with a trade-off between accuracy and speed. 

Classification accuracy on previously unseen images (in the validation folder) is a good proxy for how the classifier would perform in the real world. Ideally, the more training images, the better the learning will be. And, the more validation images, the better our classifier would perform in the real-world.

In [ ]:
#!ls | grep cat | sort -R | head -250 | xargs -I {} mv {} train/cat/
#!ls | grep dog | sort -R | head -250 | xargs -I {} mv {} train/dog/
#!ls | grep cat | sort -R | head -250 | xargs -I {} mv {} val/cat/
#!ls | grep dog | sort -R | head -250 | xargs -I {} mv {} val/dog/

## Set up the configuration

Let's start off with our Python program and begin with importing the necessary packages.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math

Let's place all the configurations up-front. These can be modified in the future based on the dataset of your choice.

In [ ]:
TRAIN_DATA_DIR = path_data + 'train/'
TRAIN_DATA_DIR

'/content/gdrive/My Drive/data/dog_cat/train/'

In [ ]:
TRAIN_DATA_DIR = path_data + 'train/'
VALIDATION_DATA_DIR = path_data + 'val/'
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

## Load and augment the data

Colored images usually have 3 channels viz. red, green and blue, each with intensity value ranging from 0 to 255. To normalize it (i.e. bring the value between 0 to 1), we can rescale the image by dividing each pixel by 255. Or, we can use the default `preprocess_input` function in Keras which does the preprocessing for us.

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

Time to load the data from its directories and let the augmentation happen! 

A few key things to note:

- Training one image at a time can be pretty inefficient, so we can batch them into groups. 
- To introduce more randomness during the training process, we’ll keep shuffling the images in each batch.
- To bring reproducibility during multiple runs of the same program, we’ll give the random number generator a seed value.

In [ ]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


Now that the data is taken care of, we come to the most crucial component of our training process - the model. We will reuse a CNN previously trained on the ImageNet dataset, remove the ImageNet specific classifier in the last few layers, and replace it with our own classifier suited to our problem. For transfer learning, we’ll ‘freeze’ the weights of the original model, i.e. set those layers as unmodifiable, so only the layers of the new classifier (that we add) can be modified. To keep things fast, we’ll choose the MobileNet model. Don’t worry about the specific layers, we’ll dig deeper into those details in [Chapter 4](https://learning.oreilly.com/library/view/practical-deep-learning/9781492034858/ch04.html).

## Define the model

In [ ]:
def model_maker():
    base_model = MobileNet(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

## Train and test


In [ ]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model.fit(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

17227776/17225924 [==============================] - 0s 0us/step
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
8/8 [==============================] - 387s 48s/step - loss: 0.5332 - acc: 0.7398 - val_loss: 0.0915 - val_acc: 0.9707
Epoch 2/10
8/8 [==============================] - 91s 11s/step - loss: 0.2089 - acc: 0.9160 - val_loss: 0.0344 - val_acc: 0.9863
Epoch 3/10
8/8 [==============================] - 49s 6s/step - loss: 0.1624 - acc: 0.9473 - val_loss: 0.0545 - val_acc: 0.9785
Epoch 4/10
8/8 [==============================] - 22s 3s/step - loss: 0.1270 - acc: 0.9611 - val_loss: 0.0357 - val_acc: 0.9863
Epoch 5/10
8/8 [==============================] - 16s 2s/step - loss: 0.1567 - acc: 0.9365 - val_loss: 0.0494 - val_acc: 0.9824
Epoch 6/10
8/8 [==============================] - 11s 1s/step - loss: 0.0899 - acc: 0.9611 - val_loss: 0.0415 - val_acc: 0.9844
Epoch 7/10
8/8 [==============================] - 12s 2s/step - loss: 0.0735 - acc: 0.96

On our runs, it took was 5 seconds in the very first epoch to reach 90% accuracy on the validation set, with just 500 training images. Whoa! And by the 10th step, we observe about 97% validation accuracy. That’s the power of transfer learning. 

Without having the model previously trained on ImageNet, getting a decent accuracy on this task would have taken (1) training time anywhere between a couple of hours to a few days (2) tons of more data to get decent results.

Before we forget, save the model you trained.

In [ ]:
model.save('model.h5')

## Model Prediction

Now that you have a trained model, you might eventually want to use it later for your application. We can now load this model anytime and classify an image. The Keras function `load_model`, as the name suggests loads the model. 

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('model.h5')

Now let’s try loading our original sample images and see what results we get.

In [ ]:
img_path = path_data+'dog.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

[[0.00272785 0.9972722 ]]
{'cat': 0, 'dog': 1}


## Hands


In [ ]:
path_data = '/content/gdrive/My Drive/data/hands/'

In [ ]:
TRAIN_DATA_DIR = path_data + 'train/'
VALIDATION_DATA_DIR = path_data + 'val/'
TRAIN_SAMPLES = 200
VALIDATION_SAMPLES = 200
NUM_CLASSES = 26
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 4680 images belonging to 26 classes.
Found 520 images belonging to 26 classes.


In [ ]:
def model_maker():
    base_model = MobileNet(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [ ]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model.fit(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

Epoch 1/10
4/4 [==============================] - 208s 52s/step - loss: 3.7207 - acc: 0.0391 - val_loss: 3.2696 - val_acc: 0.1094
Epoch 2/10
4/4 [==============================] - 73s 18s/step - loss: 3.3290 - acc: 0.0625 - val_loss: 3.1038 - val_acc: 0.1875
Epoch 3/10
4/4 [==============================] - 68s 17s/step - loss: 3.1925 - acc: 0.0781 - val_loss: 3.0414 - val_acc: 0.1602
Epoch 4/10
4/4 [==============================] - 67s 17s/step - loss: 3.1212 - acc: 0.0703 - val_loss: 2.9139 - val_acc: 0.1797
Epoch 5/10
4/4 [==============================] - 64s 16s/step - loss: 3.0036 - acc: 0.1484 - val_loss: 2.8644 - val_acc: 0.1367
Epoch 6/10
4/4 [==============================] - 54s 13s/step - loss: 2.9669 - acc: 0.1250 - val_loss: 2.7418 - val_acc: 0.2148
Epoch 7/10
4/4 [==============================] - 50s 13s/step - loss: 2.8942 - acc: 0.1172 - val_loss: 2.6111 - val_acc: 0.3711
Epoch 8/10
4/4 [==============================] - 57s 14s/step - loss: 2.9319 - acc: 0.1250 - va

In [ ]:
model.save('model.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('model.h5')

In [ ]:
img_path = path_data+'A538.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

[[0.03139458 0.0719717  0.02074774 0.0220159  0.04696643 0.01447384
  0.03568966 0.0185008  0.03574303 0.03013502 0.02562579 0.02978063
  0.09543489 0.04789203 0.01826583 0.02731701 0.00571925 0.06339716
  0.02389185 0.02692734 0.05205129 0.11559157 0.03889422 0.02692612
  0.04007148 0.03457489]]
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}


In [ ]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model.fit(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
    epochs=100,
    validation_data=validation_generator,
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

Epoch 1/100
4/4 [==============================] - 11s 3s/step - loss: 3.8321 - acc: 0.0312 - val_loss: 3.1207 - val_acc: 0.1055
Epoch 2/100
4/4 [==============================] - 10s 3s/step - loss: 3.2279 - acc: 0.1133 - val_loss: 3.0212 - val_acc: 0.2031
Epoch 3/100
4/4 [==============================] - 11s 3s/step - loss: 3.1194 - acc: 0.0977 - val_loss: 2.9075 - val_acc: 0.1211
Epoch 4/100
4/4 [==============================] - 11s 3s/step - loss: 3.0952 - acc: 0.1016 - val_loss: 2.8641 - val_acc: 0.2266
Epoch 5/100
4/4 [==============================] - 9s 2s/step - loss: 3.0631 - acc: 0.0898 - val_loss: 2.8178 - val_acc: 0.2578
Epoch 6/100
4/4 [==============================] - 6s 2s/step - loss: 3.0146 - acc: 0.1500 - val_loss: 2.6906 - val_acc: 0.3594
Epoch 7/100
4/4 [==============================] - 11s 3s/step - loss: 2.9734 - acc: 0.1328 - val_loss: 2.6022 - val_acc: 0.3359
Epoch 8/100
4/4 [==============================] - 9s 2s/step - loss: 2.9234 - acc: 0.1836 - val_lo

In [ ]:
model.save('model.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('model.h5')

In [ ]:
img_path = path_data+'A538.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

[[6.9541276e-01 4.2292699e-03 2.3507679e-02 4.3859567e-02 1.9467544e-02
  6.3630432e-02 2.5264038e-02 4.6668857e-04 2.5347039e-02 1.0730526e-03
  2.4110663e-03 3.7078774e-03 3.7987221e-03 1.7847326e-02 1.2594777e-03
  5.6591078e-05 2.4122624e-05 2.8368246e-04 8.9046825e-03 1.8888319e-04
  7.4729149e-04 4.8159263e-03 2.2629825e-03 6.4583967e-04 5.0580941e-02
  2.0657617e-04]]
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
